# Handling TCGA data
## Your input
- Filter and select TCGA data from the TCGA GDC data portal as explained in ["TCGA_steps_explained.ipynb"](TCGA_steps_explained.ipynb)
- Follow these steps every time for your new analyses, also when you have new aspects or file types to consider later on
- Create a folder called "sample_sheets" in your analysis path, create the folders "manifests" and "sample_sheets_prior" in the "sample_sheets" folder
- Adapt the ["data/config.yaml"](data/config.yaml)

## Check validity of configuration file entries

In [43]:
# Check validity of important configuration file entries
import yaml
import os
import subprocess
import shlex

with open('data/config.yaml', 'r') as streamfile:
    config_file = yaml.load(streamfile, Loader=yaml.FullLoader)

# Check analysis path
analysis_path = config_file['analysis_path']
# if analysis_path is valid
if not os.path.exists(analysis_path):
    print('Please make sure the analysis path exists and includes the manifest files. Your current analysis path is the following:')
    print(analysis_path)
else:
    # if analysis_path has "/" at the end
    if analysis_path[-1] != '/':
        analysis_path = analysis_path+'/'
        change_config_cmd = f"sed -i '/^analysis_path: .*/s/$/\//' data/config.yaml"
        subprocess.run(shlex.split(change_config_cmd))
    # if sample sheet and manifests folders exist
    if not os.path.exists(analysis_path+'sample_sheets/manifests') or not os.path.exists(analysis_path+'sample_sheets/sample_sheets_prior'):
        print('Please make sure the paths sample_sheets/manifests and sample_sheets/sample_sheets_prior exist in your analysis folder.')
    else:
        # if files in manifest and sample sheet folders exist
        manifests_prior = config_file['manifests_prior']
        sample_sheets_prior = config_file['sample_sheets_prior']

        for category_list, category_name in zip([manifests_prior, sample_sheets_prior], ['manifests', 'sample_sheets_prior']):
            for check_file in category_list:
                if not os.path.isfile(analysis_path+f'sample_sheets/{category_name}/'+check_file):
                    print(f'{check_file} ist not existent. Please check again.')
                else:
                    continue
        
        # if sample sheet for filtering is existent
        sample_sheet_filtering = config_file['sample_sheet_filtering']

        if sample_sheet_filtering != False:
            if not os.path.isfile(sample_sheet_filtering):
                print(f'{sample_sheet_filtering} is not existent. Please check again or input: False')
        
        # if manifest for download is existent
        manifest_for_download = config_file['manifest_for_download']

        if manifest_for_download != False:
            for check_file in manifest_for_download:
                if not os.path.isfile(analysis_path+f'sample_sheets/manifests/'+check_file):
                    print(f'{check_file} is not existent. Please check again or input: False')

# Check if TCGA user token file is existent
tcga_user_token_file = config_file['tcga_user_token_file']
if tcga_user_token_file != False:
    if not os.path.isfile(tcga_user_token_file):
        print(f'User token file not existent, please check again or if you do not want to use one, input: False. Your current analysis path is the following:')
        print(tcga_user_token_file)


## Combine manifest with sample sheet, filter for relevant files

- Merge manifest and sample sheet
- If previous selection of case IDs -> filter for specific case IDs of previous analysis
- Create adapted filtered manifest files for gdc-client download, rename them with the suffix "_ready.txt"

In [144]:
import pandas as pd
import yaml

# load config file
with open('data/config.yaml', 'r') as streamfile:
    config_file = yaml.load(streamfile, Loader=yaml.FullLoader)

# analysis path
analysis_path = config_file['analysis_path']

# original (prior) manifest files, merge them
manifests = config_file['manifests_prior']
manifests_dfs = [pd.read_table(analysis_path+'sample_sheets/manifests/'+i) for i in manifests]

# original (prior) sample sheets, merge them
sample_sheets = [pd.read_table(analysis_path+'sample_sheets/sample_sheets_prior/'+i) for i in config_file['sample_sheets_prior']]
sample_sheets_merge = pd.concat(sample_sheets)

sample_sheets_merge['Case ID'] = sample_sheets_merge['Case ID'].str.split(', ', expand=True)[0]

# if filtering of manifest file on case IDs of previous sample sheet is wanted
sample_sheet_filtering = config_file['sample_sheet_filtering']

if sample_sheet_filtering != False:
    sample_sheet_filter = pd.read_table(analysis_path+'sample_sheets/'+sample_sheet_filtering)
    case_ids = list(sample_sheet_filter['Case ID'].unique())
    for manifest, file_name_manifest in zip(manifests_dfs, manifests):
        filtered_manifest = manifest[(manifest['Case ID'].isin(case_ids))&(manifest['Sample Type']=='Primary Tumor')].copy()
        filtered_manifest.to_csv((analysis_path+'sample_sheets/manifests/'+manifest[0]+'_ready'+'.txt'), sep='\t', index=False)
else:
    for manifest, file_name_manifest in zip(manifests_dfs, manifests):
        manifest.to_csv((analysis_path+'sample_sheets/manifests/'+manifest[0]+'_ready'+'.txt'), sep='\t', index=False)


# change config.yaml "manifest_for_download" with manifests file name rsplit('.',1)[0] + _ready + manifests file name rsplit('.',1)[0]

# manifests for download
import shlex
import subprocess
import yaml

with open('data/config.yaml', 'r') as streamfile:
    config_file = yaml.load(streamfile, Loader=yaml.FullLoader)

# filtered manifest(s) in there, but only names in 'sample_sheets/manifests/
if config_file['manifest_for_download'] == False:
    manifests_pipeline = ', '.join(manifests_pipeline_files) # only file names
    
    manifest_file_change_cmd = f"sed -i 's/^manifest_for_download: .*/manifest_for_download: [{manifests_pipeline}]/' data/config.yaml"
    subprocess.run(shlex.split(manifest_file_change_cmd))

### Download TCGA data via a manifest document and the GDC-client tool
- This part creates a new conda environment called "gdc_client" and downloads the gdc-client tool
- Download of the TCGA data from manifest specified in previous steps and/or the configuration file via the gdc-client
- The files from the manifest are downloaded into the following folder: analysis_path + '00_raw_data'

- Optional: for restricted access files:
    - Login at NIH for restricted access files
    - Download access token, save as secured file

- 

```
gdc-client download -m manifest.txt -t user-token.txt
```


In [106]:
import os
import subprocess
import yaml
import shlex

# Read in manifest download files (either from manual input in config.yaml or from previous pipeline steps)
def Create_Manifest_Download_List():
    with open('data/config.yaml', 'r') as streamfile:
        config_file = yaml.load(streamfile, Loader=yaml.FullLoader)
    
    manifest_download_list = config_file['manifest_for_download']
    analysis_path = config_file['analysis_path']

    if manifest_download_list == False:
        manifest_for_download = False
        print('Please execute the previous part of the pipeline or input your files manually in the data/config.yaml file')
    else:
        manifest_for_download = [analysis_path+'sample_sheets/manifests/'+i for i in manifest_download_list]
    
    return manifest_for_download


# Check whether files in manifest have already been downloaded an exclude them from the manifest
def Check_Already_Downloaded():
    with open('data/config.yaml', 'r') as streamfile:
        config_file = yaml.load(streamfile, Loader=yaml.FullLoader)
    
    analysis_path = config_file['analysis_path']
    
    manifest_for_download_prior = Create_Manifest_Download_List()

    manifest_for_download = []

    if manifest_for_download_prior != False:
        for manifest_file in manifest_for_download_prior:
            manifest = pd.read_table(manifest_file).drop_duplicates()
            manifest['file_path'] = analysis_path + '00_raw_data/' + manifest['id'] + '/' + manifest['filename']
            manifest['is_file'] = manifest['file_path'].apply(os.path.isfile)
            manifest2 = manifest[manifest['is_file']==False].copy()
            manifest_file_name_2 = manifest_file.rsplit('.', 1)[0] + '_check' + manifest_file.rsplit('.', 1)[1]
            manifest2.to_csv(manifest_file_name_2)
            manifest_for_download.append(manifest_file_name_2)
    
    return manifest_for_download


# Download the gdc-client in a new conda environment and run the gdc-client, if accepted
def Download_gdc_client():
    with open('data/config.yaml', 'r') as streamfile:
        config_file = yaml.load(streamfile, Loader=yaml.FullLoader)
    
    conda_gdc = config_file['conda_gdc']
    name_conda_gdc_env = False

    if conda_gdc == False:
        print('Please execute the TCGA data download in your own environment or '+
            'set "conda_gdc" in the data/config.yaml file to "True" to create a conda environment with the gdc-client')
    elif conda_gdc == True:
        name_conda_gdc_env = 'gdc_client'
    elif conda_gdc == 'First_install':
        gdc_client_conda_cmd = shlex.split(f'conda create --name gdc_client --file envs/gdc_client.txt')
        subprocess.run(gdc_client_conda_cmd)
        
        conda_gdc_status_change_cmd = f"sed -i 's/^conda_gdc: .*/conda_gdc: True/' data/config.yaml"
        subprocess.run(shlex.split(conda_gdc_status_change_cmd))

        name_conda_gdc_env = 'gdc_client'
    else:
        name_conda_gdc_env = conda_gdc
    
    return name_conda_gdc_env


# Prepare commands to download TCGA data, dependent on available user token file
def TCGA_Data_Download():
    with open('data/config.yaml', 'r') as streamfile:
        config_file = yaml.load(streamfile, Loader=yaml.FullLoader)
    
    tcga_user_token_file = config_file['tcga_user_token_file']
    analysis_path = config_file['analysis_path']
    raw_data_path = analysis_path + '00_raw_data'

    os.makedirs(raw_data_path, exist_ok=True)

    manifest_for_download = Check_Already_Downloaded()

    name_conda_gdc_env = Download_gdc_client()

    if manifest_for_download == False:
        print('No TCGA data are download due to no manifest files.')
    elif name_conda_gdc_env == False:
        print('No TCGA data are downloaded due to the specifications in the gdc client conda environment.')
    else:
        for manifest_file in manifest_for_download:
            if tcga_user_token_file == False:
                print(f'Download TCGA data with TCGA manifest {manifest_file.split("/")[-1]} without TCGA user token')
                command_download_tcga_data = f'conda run -n {name_conda_gdc_env} gdc-client download -m {manifest_file}'
            else:
                print(f'Download TCGA data with TCGA manifest {manifest_file.split("/")[-1]} with TCGA user token file {tcga_user_token_file.split("/")[-1]}')
                command_download_tcga_data = f'conda run -n {name_conda_gdc_env} gdc-client download -m {manifest_file} {tcga_user_token_file}'
            process = subprocess.Popen(command_download_tcga_data, cwd=raw_data_path, shell=True)
            process.wait()

TCGA_Data_Download()

### Rename file names with symlinks
- In manifest only id, filename with 36 different characters
- Merge manifest and sample sheet, create new names 
- Rename downloaded files and put them in new folders for each analysis


In [149]:
import os
import pandas as pd
import yaml

with open('data/config.yaml', 'r') as streamfile:
    config_file = yaml.load(streamfile, Loader=yaml.FullLoader)

analysis_path = config_file['analysis_path']
os.makedirs(analysis_path+'01_sample_data', exist_ok=True)

downloaded_manifests = config_file['manifest_for_download']
downloaded_manifests = pd.concat([pd.read_table(analysis_path+'sample_sheets/manifests/'+i.rsplit('.', 1)[0]+'_ready'+i.rsplit('.', 1)[1]) for i in downloaded_manifests])
downloaded_manifests = downloaded_manifests.rename(columns={'id':'File ID', 'filename':'File Name'})

# original (prior) sample sheets, merge them
sample_sheets = [pd.read_table(analysis_path+'sample_sheets/sample_sheets_prior/'+i) for i in config_file['sample_sheets_prior']]
sample_sheets_merge = pd.concat(sample_sheets)

sample_sheets_merge['Case ID'] = sample_sheets_merge['Case ID'].str.split(', ', expand=True)[0]

# categorize samples for their analysis method
method_dict = {'BRASS':'BRASS', 'CaVEMan':'CaVEMan', 'ASCAT':'CNV_segment', 'pindel':'Pindel', 'star_splice':'Splicing', 
               'star_gene_counts':'STAR_counts'}

sample_sheets_merge['Folder'] = ''

for met in method_dict.keys():
    sample_sheets_merge.loc[sample_sheets_merge['File Name'].str.contains(met), 'Folder'] = method_dict[met]

# 36 characters file ID
sample_sheets_merge['File Suffix'] = sample_sheets_merge['File Name'].str[36:]

sample_sheets_merge['Path_raw'] = analysis_path+'00_raw_data/'+sample_sheets_merge['File ID']+'/'+sample_sheets_merge['File Name']
sample_sheets_merge['Path_sample'] = analysis_path+'01_sample_data/'+sample_sheets_merge['Folder']+'/'+sample_sheets_merge['Case ID']+sample_sheets_merge['File Suffix']

# merge with executed manifest file(s) on executed manifest files
downloaded_manifests = downloaded_manifests.merge(sample_sheets_merge, how='left', on=['File ID', 'File Name'])

# create folders for methods
methods = list(downloaded_manifests['Folder'].unique())
for m in methods:
    os.makedirs(analysis_path+'01_sample_data/'+m, exist_ok=True)

# create symlinks for samples to categorize them in 
for raw_path, sample_path in zip(list(downloaded_manifests['Path_raw']), list(downloaded_manifests['Path_sample'])):
    try:
        os.symlink(raw_path, sample_path)
    except FileExistsError:
        print(sample_path.split('/')[-1] + ' already exists.')
    except FileNotFoundError:
        print(sample_path.split('/')[-1] + ' not found.')


### Analyze files
- with Snakemake pipeline


In [ ]:
import subprocess
import yaml
import shlex

with open('data/config.yaml', 'r') as streamfile:
    config_file = yaml.load(streamfile, Loader=yaml.FullLoader)

conda_snakemake = config_file['conda_snakemake']
snakemake_threads = config_file['snakemake_threads']

# run Snakemake pipeline
command_snakemake = f'conda run -n {conda_snakemake} snakemake --codes {snakemake_threads} --use-conda'
process_snakemake = subprocess.Popen(command_snakemake, shell=True)
process_snakemake.wait()
